<a href="https://colab.research.google.com/github/bominwang/Bayesian-statistics-method/blob/bayesian-model-averaging-method/bayesian_model_averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np

In [23]:
def error_covariance(experiment_data, model_data):
  # experiment_data 和 model_data 都为numpy数组，每行表示一个样本
  data_volume = experiment_data.shape[0]
  covariance_matrix = np.divide(np.dot((experiment_data - model_data), (experiment_data - model_data).T), data_volume)
  return covariance_matrix